In [ ]:


import os
import cv2
import numpy as np
from tqdm import tqdm

# training directories
TRAININ_DIR = 'cuhksketches'
TRAINOUT_DIR = 'CUHKphotos'

#function to create and save training data and resize it to 40*40 pixels
def create_train_data():
    i = 0
    training_output = []
    training_input = []
    for img in tqdm(os.listdir(TRAININ_DIR)):
        path = os.path.join(TRAININ_DIR,img)
        img = cv2.imread(path)
        img = cv2.resize(img, (40,40))
        training_input.append([np.array(img)])
        
       
    
    for img in tqdm(os.listdir(TRAINOUT_DIR)):
        path = os.path.join(TRAINOUT_DIR,img)
        img1 = cv2.imread(path)
        img1 = cv2.resize(img1, (40,40))
        training_output.append([np.array(img1)])
        
        
    
    
   
    np.save('training_input_cuhk.npy', training_input)
    np.save('training_output_cuhk.npy', training_output)
    
    return training_input

In [ ]:
train_data = create_train_data()

In [ ]:
import numpy as np

#load saved training data if using in the future
training_input = np.load('training_input_cuhk.npy')
training_output = np.load('training_output_cuhk.npy')

In [ ]:
#splitting training input and output into training and validationg sets to correct sizes

X = training_input[:180].reshape(-1,40,40,3)
Y = training_output[:180].reshape(-1,40,40,3)

test_x = training_input[180:].reshape(-1,40,40,3)
test_y = training_output[180:].reshape(-1,40,40,3)



In [ ]:
import tensorflow as tf
tf.reset_default_graph()


import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d, conv_2d_transpose, residual_block
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import batch_normalization

convnet = input_data(shape=[None, 40, 40, 3], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
##convnet = batch_normalization(convnet)
##convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = conv_2d(convnet, 32, 5, activation='relu')
##convnet = max_pool_2d(convnet, 5)

##convnet = conv_2d(convnet, 64, 5, activation='relu')
##convnet = conv_2d(convnet, 32, 5, activation='relu')
##convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = conv_2d(convnet, 64, 5, activation='relu')

##convnet = batch_normalization(convnet)
##convnet = conv_2d(convnet, 32, 5, activation='relu')
##convnet = max_pool_2d(convnet, 5)

##convnet = conv_2d(convnet, 64, 5, activation='relu')

##convnet = residual_block(convnet, 3, 128, activation='relu')
##convnet = batch_normalization(convnet)
##convnet = residual_block(convnet, 3, 128, activation='relu')
##convnet = batch_normalization(convnet)
##convnet = residual_block(convnet, 3, 128, activation='relu')

##convnet = conv_2d_transpose(convnet, 8, 3,[96,96], activation='relu')
##convnet = batch_normalization(convnet)


convnet = dropout(convnet, 0.8)

convnet = conv_2d(convnet,3, 5, activation='linear')
##convnet = batch_normalization(convnet)
convnet = regression(convnet, optimizer='sgd', learning_rate=0.001, loss='mean_square', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


model.fit({'input': X}, {'targets': Y}, n_epoch=200, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=5, show_metric=True)

In [ ]:
#predict output model for a given test data and save

import cv2
import matplotlib.pyplot as plt
%matplotlib inline

test_data = X[10]

cv2.imwrite('143one.png', test_data)
np.shape(test_data)
model_out = model.predict([test_data])


x = np.reshape(model_out, (40,40,3))

cv2.imwrite('143.png', x)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
#link to view tensorboard plots in python

tensorboard --logdir=foo:C:\log